This file is for setting up the individual Rook classes so that they work seemlessly together

In [176]:
class Board:
    def __init__(self, initial_state = None, bounds = None):
        """ The board constructor initializes the current move set and game 
        state.

        Args:
            initial_state : The initial game state
        Return:
            A new and initialized Board instance
        """
        self.initial_state = initial_state
        self.state = initial_state
        self.bounds = bounds
        self.data = {}
    
    def reset():
        """ Resets the board to its original state.
        """
        self.state = self.initial_state
        self.data = {}

In [177]:
class Player:
    def __init__(self, name, strategy):
        """ The player constructor

        Args:
            name: the player's label
            strategy: a strategy function used by the player

        Return:
            An initialized instance of a Player
        """
        self.name = name
        self.strategy = strategy

    def move(self, board):
        """ This function takes in a board and then allows the player to make
        a choice on the next move.

        Args:
            board : the board object for the current game

        Return:
            The move made.
        """
        return self.strategy.move(board)

In [178]:
class Referee:
    def __init__(self, board, rules):
        """ This class models a referee for a game.
        Args:
            board : The board of the game being played``

        """
        self.board = board
        self.rules = rules

    def update_board(self, board, player, move):
        """ This method will handle updating the game board.

        Args:
            board : the game board
            player : the current player making the move
            other_player : the player not making the move

        Return:
            The winning_player or None depending on the move update

        """
        board.data[board.state] = (player.name, move)
        self.rules.update_board(board, move)

    def ask_for_move(self, player, opponent, board):
        """ This method takes in the board and current player and prompts for
        the next move.

        Then returns the move made, or None if the player did not/cannot move legally

        Args:
            player : player whose turn it is
            opponent: player whose turn it is not
            board : the game board

        Return:
            The players move
        """

        # Ask for a move from the player
        proposed_move = player.move(board)

        # Check if the player has given up or if the move is illegal
        if proposed_move is None or not self.rules.is_legal(self.board, proposed_move):
            return None
        
        # Otherwise, the move was valid, so return the proposed move
        else:
            return proposed_move
        
    def is_game_over(self, board, player):
        return self.rules.is_game_over(board, player)

In [220]:
class Game:
    def __init__(self, referee, board, players):
        """The game constructor declares a new board, referee, and two
        players. First player should be listed first. 

        Args:
            referee : the current game referee
            board : the game board
            players : the list of players
        """
        self.referee = referee
        self.board = board
        self.player = players[0]
        self.opponent = players[1]
        

    def play(self):
        """ This function allows for the game to be played.

        Args:
            None

        Return:
            winner (player_name): the name of the player who won the game 
        """
        
        winner = None
        count = 0
        while winner is None:            
            print("State: " + str(board.state))

            # Request a move from the current player
            move = self.referee.ask_for_move(self.player, self.opponent, self.board)
            print(self.player.name + ": " + str(move))
            
            # If the move is NOT valid, exit the loop
            if move is None:
                winner = self.opponent
                self.referee.is_game_over(self.board, self.player)
                print("LOSER")
                break;
            
            # Tell the referee to update the board with the (valid) move
            self.referee.update_board(self.board, self.player, move)
            
            # Check if the game has been won
            if self.referee.is_game_over(self.board, self.player):
                winner = self.player
                print("WINNER FOUND")
            
            # Next turn
            self.player, self.opponent = self.opponent, self.player
            
            # Safety check so the game doesnt run infinitely
            count += 1
            if count == 100:
                print("AAA")
                break
            
                
        return self.board

In [221]:
class Ruleset:
    def __init__(self, name):
        self.name = name
    
    def is_legal(self, board, proposed_move):
        # Move is legal if it is greater than 0 and less than or equal to the number of toothpicks left or 2, whichever is smaller
        return proposed_move > 0 and proposed_move <= min(2, board.state)
    
    def is_game_over(self, board, player):
        if board.state == 0:
            board.data["winner"] = player.name
            return True
    
    def update_board(self, board, move):
        board.state -= move

In [222]:
import random
class Strategy:
    def __init__(self, name, data = None):
        self.name = name
    
    def move(self, board):
        return random.randrange(1, 3)

In [223]:
rule = Ruleset("Basic Game Ruleset")

strat = Strategy("Random guess strategy")

p1 = Player("Player1", strat)
p2 = Player("Player2", strat)
board = Board(10)
ref = Referee(board, rule)
game = Game(ref, board, [p1, p2])
b = game.play()

State: 10
Player1: 2
State: 8
Player2: 2
State: 6
Player1: 1
State: 5
Player2: 2
State: 3
Player1: 1
State: 2
Player2: 2
WINNER FOUND


In [224]:
b.data

{10: ('Player1', 2),
 8: ('Player2', 2),
 6: ('Player1', 1),
 5: ('Player2', 2),
 3: ('Player1', 1),
 2: ('Player2', 2),
 'winner': 'Player2'}